In [201]:
import tensorflow as tf
import gym
from collections import deque
import random
import numpy as np

In [202]:
# 環境構築
env = gym.make('MountainCarContinuous-v0')

state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

print( state_dim, act_dim )

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
2 1


In [203]:
# actor
tf.reset_default_graph()

# 観測
a_state = tf.placeholder(tf.float32, [None, state_dim])

# Criticから渡される勾配
a_action_grads = tf.placeholder(tf.float32, [None, act_dim])

# ネットワーク構築
with tf.variable_scope("actor"):
    fc1 = tf.contrib.layers.fully_connected(a_state, 400 )
    fc2 = tf.contrib.layers.fully_connected(fc1, 300)
    fc3 = tf.contrib.layers.fully_connected(fc2, act_dim, activation_fn=tf.tanh )
    a_output = tf.multiply(fc3, 1)

# 勾配を計算
a_params = [ v for v in tf.trainable_variables() if "actor" in v.name ]
a_grads = tf.gradients(a_output, a_params, -a_action_grads)
a_grads = list(map(lambda x: tf.div(x, 20),  a_grads))

# 最適化
a_train_step = tf.train.AdamOptimizer(0.0001).apply_gradients(zip(a_grads,  a_params))

In [204]:
# critic
c_state = tf.placeholder(tf.float32, [None, state_dim])
c_action = tf.placeholder(tf.float32, [None, act_dim])
c_target_q = tf.placeholder(tf.float32, [None, act_dim])

fc1 = tf.contrib.layers.fully_connected(c_state, 400)
w1 = tf.Variable(tf.random_normal([400, 300], mean=0.0, stddev=0.05))
w2 = tf.Variable(tf.random_normal([act_dim, 300], mean=0.0, stddev=0.05))
b2 = tf.Variable(tf.zeros([300]))

# 二層目でactionと結合
fc2 = tf.nn.relu(tf.matmul(fc1, w1) + tf.matmul(c_action, w2) + b2)
c_output = tf.contrib.layers.fully_connected(fc2, act_dim, activation_fn=None)

c_loss = tf.reduce_mean(tf.square(c_target_q - c_output))

#c_loss = tf.losses.huber_loss(c_target_q, c_output)
c_train_step = tf.train.AdamOptimizer(0.001).minimize(c_loss)
c_action_grads = tf.gradients(c_output, c_action)


In [205]:
# メモリ
memory = deque(maxlen=1000)

def sample( memory, batch_size ):
    samp = random.sample( memory, batch_size )
    
    states = []
    actions = []
    rewards = []
    next_states = []
    
    for s, a, r, ns in samp:
        states.append( s )
        actions.append( a )
        rewards.append( r )
        next_states.append( ns )
    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states)

In [206]:
memory.clear()

env.reset()
action = env.action_space.sample()
state, reward, done, _ = env.step(action)
for i in range(1, 100):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.append((state, action, reward, next_state))
    state = next_state
    if done:
        break

In [ ]:
with tf.Session() as sess:
    sess.run( tf.global_variables_initializer() )
    for ep in range(1000):
        total_reward = 0
        env.reset()
        action = env.action_space.sample()
        
        state, reward, done, _ = env.step(action)
        for _ in range(1000):
            # training
            states, actions, rewards, next_states = sample( memory, 20 )
                        
            feed_dict = {a_state: states}
            next_actions = sess.run( a_output, feed_dict=feed_dict )
            next_q = sess.run( c_output, feed_dict={c_state:states, c_action: actions} )
            
            # critic学習
            feed_dict = {
                c_state: states, 
                c_action: actions, 
                c_target_q: rewards.reshape(20,1) + 0.99 * next_q
            }
            loss, q, _ = sess.run( [c_loss, c_output, c_train_step] , feed_dict=feed_dict)
            
            # criticでacitonの勾配を計算
            feed_dict = {
                c_state: states,
                c_action: actions
            }
            action_grads = sess.run( c_action_grads, feed_dict=feed_dict )[0]
            
            # actor学習
            feed_dict = {
                a_state: states,
                a_action_grads: action_grads                
            }
            sess.run( a_train_step, feed_dict=feed_dict )
                        
            # 実際に行動
            if ep%10==0:
                env.render()
            action = sess.run( a_output, feed_dict={a_state: state.reshape(-1,state_dim)} )[0]
            
            next_state, reward, done, _ = env.step(action)
            
            memory.append( (state.flatten(), action, reward, next_state) )

            total_reward += reward
            state = next_state
            if done:
                break
        if ep%10==0:
            print(ep  ,total_reward, loss)

0 -74.05612537251453 1.07291e-05
